In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import cisco_fiscal_calendar as cfc
import compass_readouts as cr

C:\Users\phsheari\Notebooks\compass_utils\compass_readouts.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one['Readout_Date'] = step_one['Readout_Date'].apply(lambda x: pd.to_datetime(x))
C:\Users\phsheari\Notebooks\compass_utils\compass_readouts.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guidf['GUID'] = guidf['GUID'].apply(stringify_element)


In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
weeks = 9, 10, None

In [4]:
nonecount = 0

In [5]:
for item in weeks:
    if item == None:
        nonecount +=1
    
weeks_length = len(weeks) - nonecount

In [6]:
weeks_length

2

In [7]:
weekA = weeks[0]
weekB = weeks[1]
weekC = weeks[2]

In [8]:
weekA, weekB, weekC

(9, 10, None)

In [9]:
if weekC == None:
    endfilename = f'FINBI_AMERICAS_Bookings_FY22Q3_W{weekA}_W{weekB}.xlsx'
else:
    endfilename = f'FINBI_AMERICAS_Bookings_FY22Q3_W{weekA}_W{weekB}_W{weekC}.xlsx'
print(endfilename)

FINBI_AMERICAS_Bookings_FY22Q3_W9_W10.xlsx


In [10]:
def get_reporting_week(finbi_fiscal_week_id):
    return cal.loc[cal['finbi_fiscal_week_id']==finbi_fiscal_week_id, 'week_of_fiscalquarter'].values[0]

In [11]:
def convert_nan_to_int(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return int(dataframe_cell)
            except:
                return pd.NA

In [12]:
def convert_nan_to_int_to_str(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return str(int(dataframe_cell))
            except:
                return pd.NA

In [13]:
def drop_columns(df, column_list):
    for col in df.columns:
        if col in column_list:
            df.drop(columns=col, inplace=True)

In [14]:
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'
compass_file = pd.read_pickle(excel_file_path_sfdc_version)

In [15]:
# compass_file['IndexID'] = pd.NA
# compass_file['Sheet Name'] = pd.NA

In [16]:
compass_file = compass_file[['Request ID', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','GUID', 'Customer Name', 'Deal ID', 'Campaign', 'Compass Advisor','Prehash_Entity',
       'Readout_Date', 'Readout Delivery Status', 'Items Recommended','Campaign Type', 'CONCIERGE_ADVISOR_ASSIGNED','CONCIERGE_INTERNAL_READOUT_DT', ]].drop_duplicates(keep='first')

In [17]:
compass_file['Deal ID'] = compass_file.apply(lambda x: convert_nan_to_int(x['Deal ID']), axis=1)

In [18]:
compass_file['GUID'] = compass_file.apply(lambda x: convert_nan_to_int(x['GUID']), axis=1)

In [19]:
compass_file = compass_file.loc[~compass_file['Readout Delivery Status'].isin(['Declined','Postponed']) | (compass_file['CONCIERGE_INTERNAL_READOUT_DT'].notnull())]

In [20]:
today = dt.datetime.today() #.isoformat()[:10]

In [21]:
ten_months_ago = today + dt.timedelta(weeks = -43, days = -3)

In [22]:
ro = cr.get_readout_dataframe()

In [23]:
ro['GUID'] = ro['GUID'].astype('Int64')

In [24]:
ro = ro.loc[ro['Readout_Date'] > ten_months_ago]

In [25]:
compass_file

,Request ID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,GUID,Customer Name,Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,118490,NaN,NaN,NaN,NaN,92690,MAHLE STIFTUNG GESELLSCHAFT MIT BESCHRAENKTER ...,<NA>,REACTIVE,None,MA92690,NaT,NaN,NaN,FIELD,NaN,NaT
1,118289,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-CENTRALEAST OPERATION,27610,STATE OF MICHIGAN,55183598,REACTIVE,Emma Baird,ST27610,NaT,NaN,NaN,FIELD,ebaird@cisco.com,NaT
2,118290,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,3409629,MERCHANTS FINANCIAL GROUP INC,<NA>,REACTIVE,Gloria Canales,ME3409629,NaT,NaN,NaN,FIELD,NaN,NaT
3,118291,AMERICAS,CANADA,ENTERPRISE CA,ENT WEST AREA,2122386,SUNCOR ENERGY INC,55477886,REACTIVE,Sangeetha Sampath,SU2122386,NaT,NaN,NaN,FIELD,NaN,NaT
4,118292,AMERICAS,CANADA,ENTERPRISE CA,ENT WEST AREA,9619279,TRANS CANADA PIPELINE,43718477,REACTIVE,Sangeetha Sampath,TR9619279,NaT,NaN,NaN,FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42564,107566,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,10368781,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
42565,107567,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,320903967,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
42566,107568,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,3773236,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
42567,107569,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,2134315,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [26]:
ro.drop_duplicates(keep='first', inplace=True)

<ipython-input-26-934176b8e819>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro.drop_duplicates(keep='first', inplace=True)


In [27]:
ro = pd.merge(ro, compass_file, how='left', left_on='Request ID', right_on='Request ID')

In [28]:
ro

,Request ID,GUID_x,Readout_Date_x,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,GUID_y,Customer Name,Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date_y,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,118238,11438,2022-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,118239,11438,2022-04-08,AMERICAS,US PS MARKET SEGMENT,SLED-WEST AREA,SLED-CENTRAL OPERATION,11438,TEXAS BUILDING AND PROCUREMENT COMMISSION,44358639,REACTIVE,Mark Vodka,TE11438,2022-04-08,NaN,NaN,FIELD,NaN,NaT
2,117567,444444,2022-04-12,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,444444,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
3,117069,16288,2022-04-19,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,16288,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
4,116418,4471862,2022-04-08,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,4471862,HENDRICKS,55099391,REACTIVE,Term,HE4471862,2022-04-08,Emailed Recommendations,"BCS,BRW,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13684,107566,10368781,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,10368781,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
13685,107567,320903967,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,320903967,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
13686,107568,3773236,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,3773236,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
13687,107569,2134315,2021-12-14,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,2134315,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [29]:
compass_drop_columns = ['Readout_Date_x','GUID_y']

In [30]:
drop_columns(ro, compass_drop_columns)

In [31]:
ro.rename(columns={'GUID_x':'GUID', 'Customer Name':'Compass GUID Name','Deal ID':'Compass Deal ID', 'Readout_Date_y':'Readout_Date'}, inplace=True)

In [32]:
ro.rename(columns={'Deal ID':'Compass Deal ID', 'Readout_Date_y':'Readout_Date'}, inplace=True)

In [33]:
ro.loc[ro['Readout_Date'].isnull()]

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
0,118238,11438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
6,115192,4892203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
7,115168,124163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
8,115164,124163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
9,115136,2119509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13667,107556,137738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
13668,107557,31894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
13669,107559,326960823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
13670,107560,3397110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT


In [34]:
ro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13689 entries, 0 to 13688
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Request ID                     13689 non-null  object        
 1   GUID                           13689 non-null  Int64         
 2   Sales Level 1                  7073 non-null   object        
 3   Sales Level 2                  7073 non-null   object        
 4   Sales Level 3                  7072 non-null   object        
 5   Sales Level 4                  7072 non-null   object        
 6   Compass GUID Name              7073 non-null   object        
 7   Compass Deal ID                2574 non-null   object        
 8   Campaign                       7072 non-null   object        
 9   Compass Advisor                7070 non-null   object        
 10  Prehash_Entity                 7073 non-null   object        
 11  Readout_Date   

In [35]:
ro.drop(ro.loc[ro['Readout_Date'].isnull()].index, inplace=True)

In [36]:
ro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7073 entries, 1 to 13688
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Request ID                     7073 non-null   object        
 1   GUID                           7073 non-null   Int64         
 2   Sales Level 1                  7073 non-null   object        
 3   Sales Level 2                  7073 non-null   object        
 4   Sales Level 3                  7072 non-null   object        
 5   Sales Level 4                  7072 non-null   object        
 6   Compass GUID Name              7073 non-null   object        
 7   Compass Deal ID                2574 non-null   object        
 8   Campaign                       7072 non-null   object        
 9   Compass Advisor                7070 non-null   object        
 10  Prehash_Entity                 7073 non-null   object        
 11  Readout_Date    

In [37]:
# sfdc_full_export_path = r'C:\Users\phsheari\Documents\SFDC Data\FULL EXPORT'
# services_source = pd.read_pickle(sfdc_full_export_path + r'\COMBINED.pkl')

In [38]:
# services_source.drop_duplicates(keep='first', inplace=True)

In [39]:
# services_source

In [40]:
weekday_dict = {0:1,       # 0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday
                1:2,
                2:3,
                3:4,
                4:5}

In [41]:
todays_weekday = today.weekday()

In [42]:
todays_weekday

0

In [43]:
if weekC == None:
    weeks_ago = today - dt.timedelta((weeks_length * 7)+(weekday_dict[todays_weekday]))      #Looking two weeks back + 2 days. This is normall run on a Monday, so we want that prior Saturday, 2 weeks ago
    weeks_ago = weeks_ago.isoformat()[:10]
else:
    weeks_ago = today - dt.timedelta((weeks_length * 7)+(weekday_dict[todays_weekday]))      #Looking three weeks back + 2 days. This is normall run on a Monday, so we want that prior Saturday, 3 weeks ago
    weeks_ago = weeks_ago.isoformat()[:10]

In [44]:
print(weeks_ago)

2022-03-27


In [45]:
cal = cfc.get_calendar_dataframe()

In [46]:
today.isoformat()[:10]

'2022-04-11'

In [47]:
cal.loc[cal['date']==today.isoformat()[:10], 'finbi_fiscal_week_id'].values[0]

2022093

In [48]:
start_fiscal_week = cal.loc[cal['date']==weeks_ago, 'finbi_fiscal_week_id'].values[0]
end_fiscal_week = cal.loc[cal['date']==today.isoformat()[:10], 'finbi_fiscal_week_id'].values[0]

In [49]:
print(start_fiscal_week, end_fiscal_week)

2022091 2022093


In [50]:
CONVERTERS = {'End Customer Global Ultimate ID': convert_nan_to_int, 'CAV¬†End Customer ID':convert_nan_to_int , 'CAV End Customer ID': convert_nan_to_int, 'Deal ID': convert_nan_to_int, }

In [51]:
renamefields = {'End Customer Global Ultimate Name':'GUID Name','ERP Deal ID':'Deal ID', 'New/Renewal':'Service Source',
       'End Customer Global Ultimate ID':'GUID', 'CAV End Customer Name':'CAV Name','CAV End Customer ID':'CAV ID',
       'CAV¬†End Customer ID':'CAV ID', 'CAV End Customer ID':'CAV ID', 'CA Service Bookings Net':'TCV', 'Annual Bookings Net':'ACV'}

In [52]:
fin_path = r'C:\Users\phsheari\Documents\FIN BI Data\1_AMERICAS'
entries = os.listdir(fin_path)

In [53]:
BCSlist = []

for f_name in entries:
    if f_name.startswith('Americas_AS BCS Bookings') and f_name.endswith('.xlsx'):      #'Americas_AS BCS'
        tempdf = pd.read_excel(fin_path + "\\" + f_name, converters = CONVERTERS, skiprows=3)
        BCSlist.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [54]:
bcs = pd.concat(BCSlist)
bcs['AS Offer Type']='BCS'

In [55]:
bcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8497 entries, 0 to 8496
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          0 non-null      float64
 1   Fiscal Year                         8497 non-null   int64  
 2   Fiscal Quarter ID                   8497 non-null   object 
 3   Fiscal Period ID                    8497 non-null   int64  
 4   Fiscal Week ID                      8497 non-null   int64  
 5   Finance BU/Service Category         8497 non-null   object 
 6   Finance Product Family/Allctd Svcs  8497 non-null   object 
 7   Sales Level 1                       8497 non-null   object 
 8   Sales Level 2                       8497 non-null   object 
 9   Sales Level 3                       8497 non-null   object 
 10  Sales Level 4                       8497 non-null   object 
 11  End Customer Global Ultimate Name   8497 no

In [56]:
bcs.columns

Index(['Unnamed: 0', 'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category',
       'Finance Product Family/Allctd Svcs', 'Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'End Customer Global Ultimate Name',
       'End Customer Global Ultimate ID', 'CAV End Customer Name',
       'CAV End Customer ID', 'Deal ID', 'New/Renewal', 'AS Offer Type',
       'Partner Name', 'Finance Sub Group/Contract Type',
       'CX Product Portfolio', 'CX Product Category', 'CX Product',
       'Product ID', 'CA Service Bookings Net', 'Annual Bookings Net'],
      dtype='object')

In [57]:
bcs.rename(columns=renamefields, inplace=True)

In [58]:
bcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8497 entries, 0 to 8496
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          0 non-null      float64
 1   Fiscal Year                         8497 non-null   int64  
 2   Fiscal Quarter ID                   8497 non-null   object 
 3   Fiscal Period ID                    8497 non-null   int64  
 4   Fiscal Week ID                      8497 non-null   int64  
 5   Finance BU/Service Category         8497 non-null   object 
 6   Finance Product Family/Allctd Svcs  8497 non-null   object 
 7   Sales Level 1                       8497 non-null   object 
 8   Sales Level 2                       8497 non-null   object 
 9   Sales Level 3                       8497 non-null   object 
 10  Sales Level 4                       8497 non-null   object 
 11  GUID Name                           8497 no

In [59]:
bcs.head(6)

,Unnamed: 0,Fiscal Year,Fiscal Quarter ID,Fiscal Period ID,Fiscal Week ID,Finance BU/Service Category,Finance Product Family/Allctd Svcs,Sales Level 1,Sales Level 2,Sales Level 3,...,Service Source,AS Offer Type,Partner Name,Finance Sub Group/Contract Type,CX Product Portfolio,CX Product Category,CX Product,Product ID,TCV,ACV
0,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,Americas,AMERICAS_MISC,US MISC,...,NEW,BCS,UNKNOWN,RBAS,Business Critical Services,BCS 2.x,BCS 2.x,CON-RBAS-1,"-1,882.13",-627.38
1,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,Americas,AMERICAS_MISC,US MISC,...,UNKNOWN,BCS,UNKNOWN,RBAS,Business Critical Services,BCS 2.x,BCS 2.x,CON-RBAS-1,0.00,0.00
2,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,Americas,AMERICAS_MISC,US MISC,...,NEW,BCS,UNKNOWN,RBAS,Business Critical Services,BCS 2.x,BCS 2.x,CON-RBAS-1,"-4,147.52","-1,382.51"
3,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,Americas,AMERICAS_MISC,US MISC,...,NEW,BCS,UNKNOWN,CSER,Success Tracks,Level 2,Integrated Secure Operations L2,CON-CSER-OA-SEC,0.00,0.00
4,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,Americas,AMERICAS_MISC,US MISC,...,NEW,BCS,UNKNOWN,CSIA,Success Tracks,Level 2,Integrated Secure Operations L2,CON-CSIA-OA-SEC,0.00,0.00
5,NaN,2022,2022Q3,202207,2022071,ADVANCED SERVICES,AS SUBSCRIPTION,Americas,AMERICAS_MISC,US MISC,...,RENEW,BCS,UNKNOWN,RBAS,Business Critical Services,BCS 2.x,BCS 2.x,CON-RBAS-1,"-1,532.96","-1,532.96"


In [60]:
bcs.shape

(8497, 26)

In [61]:
HTEClist = []

for f_name in entries:
    if f_name.startswith('Americas_TS HTEC') and f_name.endswith('.xlsx'):
        tempdf = pd.read_excel(fin_path + "\\" + f_name, converters = CONVERTERS, skiprows=3)
        HTEClist.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [62]:
htec = pd.concat(HTEClist)
htec.rename(columns={'Finance Program Brand':'AS Offer Type'}, inplace=True)

In [63]:
htec.shape

(342534, 26)

In [64]:
def get_offer_type(cx_product_category):
    if pd.isnull(cx_product_category):
        return pd.NA
    elif cx_product_category.lower() == 'expert care':
        return 'HTEC'
    elif cx_product_category.lower() == 'solution support':
        return 'Solution Support'
    else:
        return pd.NA

In [65]:
htec['AS Offer Type'] = htec['CX Product Category'].apply(get_offer_type)

In [66]:
bcs.columns

Index(['Unnamed: 0', 'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category',
       'Finance Product Family/Allctd Svcs', 'Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID', 'CAV Name',
       'CAV ID', 'Deal ID', 'Service Source', 'AS Offer Type', 'Partner Name',
       'Finance Sub Group/Contract Type', 'CX Product Portfolio',
       'CX Product Category', 'CX Product', 'Product ID', 'TCV', 'ACV'],
      dtype='object')

In [67]:
htec.rename(columns=renamefields, inplace=True)

In [68]:
htec.columns

Index(['Unnamed: 0', 'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category',
       'Finance Product Family/Allctd Svcs', 'Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID', 'CAV Name',
       'CAV ID', 'AS Offer Type', 'Partner Name', 'Deal ID', 'Service Source',
       'Finance Sub Group/Contract Type', 'CX Product Portfolio',
       'CX Product Category', 'CX Product', 'Product ID', 'TCV', 'ACV'],
      dtype='object')

In [69]:
# bcs.drop(columns=['Unnamed: 0','Unnamed: 4','Unnamed: 8','Date Booked','Fiscal Week End Date','Finance Product Family/Allctd Svcs'], inplace=True)

In [70]:
htec.drop(columns='Unnamed: 0', inplace=True)

In [71]:
Successlist = []

for f_name in entries:
    if f_name.startswith('Americas_Success') and f_name.endswith('.xlsx'): #sheet_name = Data
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        Successlist.append(tempdf)

In [72]:
if Successlist:
    success = pd.concat(Successlist)
else:
    success = None

In [73]:
success.shape

(3425, 23)

In [74]:
if success is not None:
    success = success.loc[(success['Fiscal Week ID'] >= start_fiscal_week) & (success['Fiscal Week ID'] < end_fiscal_week)]

In [75]:
success = success.loc[success['CA Service Bookings Net']>1]

In [76]:
success.shape

(413, 23)

In [77]:
SNTC_list = []

for f_name in entries:
    if f_name.startswith('Americas_SNTC') and f_name.endswith('.xlsx'): 
        tempdf = pd.read_excel(fin_path + "\\" + f_name, skiprows=3, converters = CONVERTERS)
        SNTC_list.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [78]:
sntc = pd.concat(SNTC_list)
sntc.rename(columns={'Finance Program Brand':'AS Offer Type'}, inplace=True)

In [79]:
sntc.rename(columns=renamefields, inplace=True)

In [80]:
sntc.shape

(394871, 23)

In [81]:
dropcolumns = ['Unnamed: 0','Unnamed: 4','Unnamed: 8','Date Booked','Fiscal Week End Date','Finance Product Family/Allctd Svcs','Finance Product Sub Group Description', 'Sales Level 5',
       'Sales Level 6', 'SSS Flag', 'SPM Coverage','DE/PSSM Flag','PDE/PSSM Flag','Contract Number','Finance Sub Group/Contract Type','CX Product Portfolio','CX Product']

In [82]:
for col in success.columns:
    if col in renamefields.keys():
        success.rename(columns = {col:renamefields[col]}, inplace=True)

In [83]:
drop_columns(success, dropcolumns)

In [84]:
drop_columns(bcs, dropcolumns)

In [85]:
drop_columns(htec, dropcolumns)

In [86]:
drop_columns(sntc, dropcolumns)

In [87]:
for col in bcs.columns:
    if col in renamefields.keys():
        bcs.rename(columns={col:renamefields[col]}, inplace=True)

In [88]:
for col in htec.columns:
    if col in renamefields.keys():
        htec.rename(columns={col:renamefields[col]}, inplace=True)

In [89]:
for col in success.columns:
    if col in renamefields.keys():
        success.rename(columns={col:renamefields[col]}, inplace=True)

In [90]:
for col in sntc.columns:
    if col in renamefields.keys():
        sntc.rename(columns={col:renamefields[col]}, inplace=True)

In [91]:
success['AS Offer Type']='SuccessTracks'
success['CX Product Category']='SuccessTracks'
sntc['AS Offer Type']='SNTC'

In [92]:
bcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8497 entries, 0 to 8496
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  8497 non-null   int64  
 1   Fiscal Quarter ID            8497 non-null   object 
 2   Fiscal Period ID             8497 non-null   int64  
 3   Fiscal Week ID               8497 non-null   int64  
 4   Finance BU/Service Category  8497 non-null   object 
 5   Sales Level 1                8497 non-null   object 
 6   Sales Level 2                8497 non-null   object 
 7   Sales Level 3                8497 non-null   object 
 8   Sales Level 4                8497 non-null   object 
 9   GUID Name                    8497 non-null   object 
 10  GUID                         8497 non-null   int64  
 11  CAV Name                     8497 non-null   object 
 12  CAV ID                       8497 non-null   int64  
 13  Deal ID           

In [93]:
htec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342534 entries, 0 to 342533
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Fiscal Year                  342534 non-null  int64  
 1   Fiscal Quarter ID            342534 non-null  object 
 2   Fiscal Period ID             342534 non-null  int64  
 3   Fiscal Week ID               342534 non-null  int64  
 4   Finance BU/Service Category  342534 non-null  object 
 5   Sales Level 1                342534 non-null  object 
 6   Sales Level 2                342534 non-null  object 
 7   Sales Level 3                342534 non-null  object 
 8   Sales Level 4                342534 non-null  object 
 9   GUID Name                    342534 non-null  object 
 10  GUID                         341872 non-null  object 
 11  CAV Name                     342534 non-null  object 
 12  CAV ID                       342534 non-null  int64  
 13 

In [94]:
success.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 2355 to 3415
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  413 non-null    int64  
 1   Fiscal Quarter ID            413 non-null    object 
 2   Fiscal Period ID             413 non-null    int64  
 3   Fiscal Week ID               413 non-null    int64  
 4   Finance BU/Service Category  413 non-null    object 
 5   Sales Level 1                413 non-null    object 
 6   Sales Level 2                413 non-null    object 
 7   Sales Level 3                413 non-null    object 
 8   Sales Level 4                413 non-null    object 
 9   GUID Name                    413 non-null    object 
 10  GUID                         413 non-null    object 
 11  CAV Name                     413 non-null    object 
 12  CAV ID                       413 non-null    int64  
 13  Deal ID         

In [95]:
success = success.reindex(columns=bcs.columns)

In [96]:
success.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 2355 to 3415
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Fiscal Year                  413 non-null    int64  
 1   Fiscal Quarter ID            413 non-null    object 
 2   Fiscal Period ID             413 non-null    int64  
 3   Fiscal Week ID               413 non-null    int64  
 4   Finance BU/Service Category  413 non-null    object 
 5   Sales Level 1                413 non-null    object 
 6   Sales Level 2                413 non-null    object 
 7   Sales Level 3                413 non-null    object 
 8   Sales Level 4                413 non-null    object 
 9   GUID Name                    413 non-null    object 
 10  GUID                         413 non-null    object 
 11  CAV Name                     413 non-null    object 
 12  CAV ID                       413 non-null    int64  
 13  Deal ID         

In [97]:
htec = htec.reindex(columns=bcs.columns)

In [98]:
success.shape

(413, 21)

In [99]:
sntc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394871 entries, 0 to 394870
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Fiscal Year                  394871 non-null  int64  
 1   Fiscal Quarter ID            394871 non-null  object 
 2   Fiscal Period ID             394871 non-null  int64  
 3   Fiscal Week ID               394871 non-null  int64  
 4   Finance BU/Service Category  394871 non-null  object 
 5   Sales Level 1                394871 non-null  object 
 6   Sales Level 2                394871 non-null  object 
 7   Sales Level 3                394871 non-null  object 
 8   Sales Level 4                394871 non-null  object 
 9   GUID Name                    394871 non-null  object 
 10  GUID                         394348 non-null  object 
 11  CAV Name                     394871 non-null  object 
 12  CAV ID                       394871 non-null  int64  
 13 

In [100]:
sntc = sntc.reindex(columns=bcs.columns)

In [101]:
sntc.shape

(394871, 21)

In [102]:
df = pd.concat([bcs,htec, success, sntc]) #,success

In [103]:
df['GUID'] = df['GUID'].astype('Int64')

In [104]:
df.head(10)

,Fiscal Year,Fiscal Quarter ID,Fiscal Period ID,Fiscal Week ID,Finance BU/Service Category,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,GUID Name,...,CAV Name,CAV ID,Deal ID,Service Source,AS Offer Type,Partner Name,CX Product Category,Product ID,TCV,ACV
0,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,BERKSHIRE HATHAWAY INC,...,GEICO,100703,18333230,NEW,BCS,UNKNOWN,BCS 2.x,CON-RBAS-1,"-1,882.13",-627.38
1,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,BERKSHIRE HATHAWAY INC,...,GEICO,100703,18333230,UNKNOWN,BCS,UNKNOWN,BCS 2.x,CON-RBAS-1,0.00,0.00
2,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,BHARTI ENTERPRISES LTD,...,BHARTI ENTERPRISES LTD,115251,43987253,NEW,BCS,UNKNOWN,BCS 2.x,CON-RBAS-1,"-4,147.52","-1,382.51"
3,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,CISCO SYSTEMS INC,...,CISCO SYSTEMS INC,425967,<NA>,NEW,BCS,UNKNOWN,Level 2,CON-CSER-OA-SEC,0.00,0.00
4,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,CISCO SYSTEMS INC,...,CISCO SYSTEMS INC,425967,<NA>,NEW,BCS,UNKNOWN,Level 2,CON-CSIA-OA-SEC,0.00,0.00
5,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,CREDICORP LTD,...,CREDICORP LTD,267993,43532007,RENEW,BCS,UNKNOWN,BCS 2.x,CON-RBAS-1,"-1,532.96","-1,532.96"
6,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,GOBIERNO DE LA REPUBLICA DE COSTA RICA,...,BANCO DE COSTA RICA,1809548,43740777,RENEW,BCS,UNKNOWN,BCS 2.x,CON-RBAS-1,"-5,700.00","-5,700.00"
7,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_MISC,US MISC,US Misc - Node 4,WMATA,...,DC-WASHINGTON METROPOLITAN AREA TRANSIT AUTHOR...,174993,53025556,NEW,BCS,Thundercat Technology,BCS 2.x,CON-AS-RS-OPT,0.00,0.00
8,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_SP,ASP CSP,ASP ATT,AT&T INC,...,AT&T INC,116125,42742291,RENEW,BCS,ATT CORP,BCS 2.x,CON-AS-MP-OPT,0.00,0.00
9,2022,2022Q3,202207,2022071,ADVANCED SERVICES,Americas,AMERICAS_SP,ASP CSP,ASP ATT,AT&T INC,...,AT&T INC,116125,43721036,RENEW,BCS,ATT CORP,BCS 3.x,CON-CXO-XA-NPRE-OA,0.00,0.00


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746315 entries, 0 to 394870
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Fiscal Year                  746315 non-null  int64  
 1   Fiscal Quarter ID            746315 non-null  object 
 2   Fiscal Period ID             746315 non-null  int64  
 3   Fiscal Week ID               746315 non-null  int64  
 4   Finance BU/Service Category  746315 non-null  object 
 5   Sales Level 1                746315 non-null  object 
 6   Sales Level 2                746315 non-null  object 
 7   Sales Level 3                746315 non-null  object 
 8   Sales Level 4                746315 non-null  object 
 9   GUID Name                    746315 non-null  object 
 10  GUID                         745130 non-null  Int64  
 11  CAV Name                     746315 non-null  object 
 12  CAV ID                       746315 non-null  int64  
 13 

In [106]:
df = df.loc[(df['Fiscal Week ID']>= start_fiscal_week) & (df['Fiscal Week ID'] < end_fiscal_week)]

In [107]:
df.shape

(207350, 21)

In [108]:
df['Fiscal Week ID'].unique()

array([2022091, 2022092], dtype=int64)

In [109]:
df['ReportWeek'] = df['Fiscal Week ID'].apply(get_reporting_week)

In [110]:
new_column_list = []
for col in df.columns:
    if '\u00C2' in col:
        col = col.replace('\u00C2','').replace('\xa0',' ')
    new_column_list.append(col)

In [111]:
for i,element in enumerate(new_column_list):
    if element=='CAV End Customer ID':
        new_column_list[i] = 'CAV ID'
        
# new_column_list    

In [112]:
df.columns = new_column_list

In [113]:
df.columns

Index(['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'GUID Name', 'GUID',
       'CAV Name', 'CAV ID', 'Deal ID', 'Service Source', 'AS Offer Type',
       'Partner Name', 'CX Product Category', 'Product ID', 'TCV', 'ACV',
       'ReportWeek'],
      dtype='object')

In [114]:
df['Deal ID'] = df['Deal ID'].apply(convert_nan_to_int)

In [115]:
df['CAV ID'] = df['CAV ID'].apply(convert_nan_to_int)

In [116]:
df['GUID'] = df['GUID'].apply(convert_nan_to_int)

In [117]:
# df['Deal ID'].unique()

In [118]:
df['Deal ID'] = df['Deal ID'].fillna(value=-9999, axis = 0)

In [119]:
def squeeze_customer_name(customer_name):
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [120]:
### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName
def make_fake_guid(guid, cavid, customername):
    if pd.isnull(guid):
        if pd.isnull(cavid) or cavid == -999:
            return str(customername[:10])
        else:
            if not isinstance(cavid, int):
                cavid = cavid.split(',')
                return str(cavid[0])
    elif not isinstance(guid,int):
        guid = guid.split(',')
        return str(guid[0])
    else:
        return str(guid)

In [121]:
df['Squeezed'] = df['CAV Name'].apply(lambda x: squeeze_customer_name(x))

In [122]:
df['Prepend'] = df['Squeezed'].apply(lambda x: x[:2].upper())

In [123]:
df['FakeID'] = df.apply(lambda x: make_fake_guid(x['GUID'], x['CAV ID'], x['Squeezed']), axis=1)

In [124]:
df.loc[df['FakeID'].isnull()][['GUID','CAV ID','CAV Name', 'Squeezed','Prepend','FakeID']]

,GUID,CAV ID,CAV Name,Squeezed,Prepend,FakeID


In [125]:
df['Prehash_Entity'] = df.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [126]:
df = df[['Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Period ID',
       'Fiscal Week ID', 'ReportWeek', 'Finance BU/Service Category', 'Sales Level 1',
       'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'Partner Name',
       'Deal ID', 'AS Offer Type', 'CX Product Category', 'Product ID', 'TCV', 'ACV', 'Service Source', 'Squeezed','Prepend','FakeID', 'Prehash_Entity']]

In [127]:
df.rename(columns = {'Deal ID':'FINBI Deal ID','Partner Name':'FINBI Partner','New/Renewal':'Service Source'}, inplace=True)

In [128]:
table = pd.pivot_table(df, values = ['ACV','TCV'], index=['Sales Level 1', 'Sales Level 2',
       'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name',  
       'FINBI Deal ID', 'FINBI Partner','AS Offer Type', 'CX Product Category', 'Fiscal Quarter ID','Fiscal Week ID', 'ReportWeek','Service Source','Squeezed','Prepend','FakeID', 'Prehash_Entity'], aggfunc='sum')

In [129]:
# table

In [130]:
table.reset_index(inplace=True)

In [131]:
table.loc[table['CX Product Category'].str.lower()=='successtracks']

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Quarter ID,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV
4767,Americas,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,112118,DIGITAL REALTY TRUST INC,39588065,DIGITAL REALTY TRUST INC,54165549,INSIGHT DIRECT USA INC,...,2022Q3,2022091,9,RENEW,DIGITALREALTYTRUSTINC,DI,39588065,DI39588065,991.20,"2,486.25"
15840,Americas,CANADA,PUBLIC SECTOR CAN,PS EAST AREA,214514,UNIVERSITE DE SHERBROOKE,2118702,UNIVERSITE DE SHERBROOKE,-9999,BELL CANADA,...,2022Q3,2022092,10,RENEW,UNIVERSITEDESHERBROOKE,UN,2118702,UN2118702,"2,113.71","2,113.71"
16576,Americas,CANADA,PUBLIC SECTOR CAN,PS FED AREA,118821,GOC_SHARED SERVICES CANADA,2122268,GOVERNMENT OF CANADA,54316202,Stoneworks Technologies Inc.,...,2022Q3,2022091,9,NEW,GOCSHAREDSERVICESCANADA,GO,2122268,GO2122268,"3,066.65","15,333.24"
16919,Americas,CANADA,PUBLIC SECTOR CAN,PS WEST AREA,164843,UNIVERSITY OF THE FRASER VALLEY,2118711,UNIVERSITY OF THE FRASER VALLEY,54094411,COMPUGEN INC.,...,2022Q3,2022092,10,NEW,UNIVERSITYOFTHEFRASERVALLEY,UN,2118711,UN2118711,"4,451.76","13,355.28"
17176,Americas,CANADA,PUBLIC SECTOR CAN,PS WEST AREA,526453,RIVER EAST TRANSCONA SCHOOL DIVISION,2118901,RIVER EAST TRANSCONA SCHOOL DIVISION,53599899,BELL CANADA,...,2022Q3,2022091,9,NEW,RIVEREASTTRANSCONASCHOOLDIVISION,RI,2118901,RI2118901,"7,581.24","7,581.24"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116540,Americas,US PS Market Segment,SLED-West Area,SLED-Gold Coast Operation,100664,STATE OF CALIFORNIA,33575,STATE OF CALIFORNIA,-9999,NWN CORPORATION,...,2022Q3,2022092,10,RENEW,STATEOFCALIFORNIA,ST,33575,ST33575,"4,038.16","4,038.16"
117822,Americas,US PS Market Segment,SLED-West Area,SLED-Gold Coast Operation,1808787,ORANGE COUNTY CALIFORNIA,18153,COUNTY OF ORANGE,55481311,"EPLUS TECHNOLOGY, INC",...,2022Q3,2022091,9,NEW,ORANGECOUNTYCALIFORNIA,OR,18153,OR18153,"60,772.96","60,772.96"
117823,Americas,US PS Market Segment,SLED-West Area,SLED-Gold Coast Operation,1808787,ORANGE COUNTY CALIFORNIA,18153,COUNTY OF ORANGE,55481311,"EPLUS TECHNOLOGY, INC",...,2022Q3,2022091,9,RENEW,ORANGECOUNTYCALIFORNIA,OR,18153,OR18153,"322,606.28","322,606.28"
119777,Americas,US PS Market Segment,SLED-West Area,SLED-West By Midwest Operation,675828,WISCONSIN TECHNICAL COLLEGE SYSTEM BOARD,3957523,WISCONSIN TECHNICAL COLLEGE SYSTEM,55734127,HEARTLAND BUSINESS SYSTEMS,...,2022Q3,2022092,10,NEW,WISCONSINTECHNICALCOLLEGESYSTEMBOARD,WI,3957523,WI3957523,"67,447.46","337,237.29"


In [132]:
table.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Deal ID',
       'FINBI Partner', 'AS Offer Type', 'CX Product Category',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Service Source',
       'Squeezed', 'Prepend', 'FakeID', 'Prehash_Entity', 'ACV', 'TCV'],
      dtype='object')

In [133]:
# table.loc[table['Bookings']>=1]

In [134]:
tableP = table.loc[table['TCV']>0]

In [135]:
tableP['Ind'] = 'P'

<ipython-input-135-09d0920af551>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tableP['Ind'] = 'P'


In [136]:
tableN = table.loc[table['TCV']<=0]

In [137]:
tableN['Ind'] = 'N'

<ipython-input-137-7a79b8daff69>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tableN['Ind'] = 'N'


In [138]:
# tableN.reset_index(inplace=True) #['Deal ID']== -9999

In [139]:
tableN

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
0,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,-999,UNKNOWN,-999,UNKNOWN,-9999,UNKNOWN,...,2022091,9,NEW,UNKNOWN,UN,-999,UN-999,0.00,0.00,N
1,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,-999,UNKNOWN,-999,UNKNOWN,-9999,UNKNOWN,...,2022091,9,RENEW,UNKNOWN,UN,-999,UN-999,0.00,0.00,N
3,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,-999,UNKNOWN,370320294,GXO LOGISTICS INC,-9999,UNKNOWN,...,2022091,9,NEW,UNKNOWN,UN,370320294,UN370320294,-11.17,-11.17,N
4,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,-999,UNKNOWN,370320294,GXO LOGISTICS INC,54263806,UNKNOWN,...,2022092,10,NEW,UNKNOWN,UN,370320294,UN370320294,-0.12,-0.35,N
5,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,100652,ATLANTIC HEALTH SYSTEM INC,4463082,ATLANTIC HEALTH SYSTEM INC,53664507,UNKNOWN,...,2022092,10,NEW,ATLANTICHEALTHSYSTEMINC,AT,4463082,AT4463082,-1.69,-1.69,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125629,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Rebates,-999,UNKNOWN,-999,UNKNOWN,41262602,UNKNOWN,...,2022091,9,UNKNOWN,UNKNOWN,UN,-999,UN-999,0.00,0.00,N
125630,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Rebates,-999,UNKNOWN,-999,UNKNOWN,41435119,UNKNOWN,...,2022091,9,NEW,UNKNOWN,UN,-999,UN-999,0.00,0.00,N
125631,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Rebates,108018,UNIVERSITY OF WISCONSIN SYSTEM,23556,UNIVERSITY OF WISCONSIN SYSTEM,45651680,UNKNOWN,...,2022091,9,NEW,UNIVERSITYOFWISCONSINSYSTEM,UN,23556,UN23556,0.00,0.00,N
125632,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Rebates,140457,RUTGERS THE STATE UNIVERSITY OF NEW JERSEY,10908,RUTGERS THE STATE UNIVERSITY OF NEW JERSEY,10606604,UNKNOWN,...,2022091,9,NEW,RUTGERSTHESTATEUNIVERSITYOFNEWJERSEY,RU,10908,RU10908,0.00,0.00,N


In [140]:
### This is DROP Criteria; i.e. > -10,000 to 0

tableN.drop(tableN.loc[(tableN['TCV'] > -10000) & (tableN['TCV'] <= 0)].index, inplace=True)   #tableN.drop(((tableN['TCV'] > -20000) & (tableN['TCV'] <= 0))].index, inplace=True) 

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [141]:
tableP

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
2,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,-999,UNKNOWN,180398450,JETSON SPECIALTY MARKETING SERVICES INC,-9999,UNKNOWN,...,2022091,9,NEW,UNKNOWN,UN,180398450,UN180398450,7.40,7.40,P
7,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,103104,BT GROUP PLC,7174334,BT GROUP PLC,-9999,UNKNOWN,...,2022091,9,NEW,BTGROUPPLC,BT,7174334,BT7174334,69.73,69.73,P
8,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,107360,PILOT CORPORATION,3397110,PILOT CORPORATION,-9999,UNKNOWN,...,2022092,10,NEW,PILOTCORPORATION,PI,3397110,PI3397110,44.83,44.82,P
9,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,107914,DIEBOLD INC,12660,DIEBOLD INC,-9999,UNKNOWN,...,2022091,9,NEW,DIEBOLDINC,DI,12660,DI12660,22.82,22.82,P
10,Americas,AMERICAS_MISC,AMERICAS MISC CONTRA L3,AMERICAS MISC CONTRA L4,107914,DIEBOLD INC,12660,DIEBOLD INC,-9999,UNKNOWN,...,2022092,10,NEW,DIEBOLDINC,DI,12660,DI12660,81.94,81.95,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125190,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Channel Programs,5390554,SOPAC PROPERTIES LLC,149451255,SOPAC PROPERTIES LLC,44198843,UNKNOWN,...,2022092,10,RENEW,SOPACPROPERTIESLLC,SO,149451255,SO149451255,"2,072.44","2,072.44",P
125193,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Channel Programs,6116241,VANDERBILT UNIVERSITY MEDICAL CENTER (VUMC),12616,THE VANDERBILT UNIVERSITY,-9999,UNKNOWN,...,2022092,10,NEW,VANDERBILTUNIVERSITYMEDICALCENTER(VUMC),VA,12616,VA12616,2.32,2.32,P
125219,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Channel Programs,7813863,CA-CITY OF GLENDALE,21452,CITY OF GLENDALE,52661138,UNKNOWN,...,2022091,9,NEW,CACITYOFGLENDALE,CA,21452,CA21452,1.13,1.13,P
125220,Americas,US PS Market Segment,US Public Sector Misc.,PS Theater Channel Programs,7813863,CA-CITY OF GLENDALE,21452,CITY OF GLENDALE,53044863,UNKNOWN,...,2022092,10,NEW,CACITYOFGLENDALE,CA,21452,CA21452,0.26,0.79,P


In [142]:
tableP['AS Offer Type'].unique()

array(['SNTC', 'Solution Support', 'BCS', 'HTEC', 'SuccessTracks'],
      dtype=object)

In [143]:
# tableP.reset_index(inplace=True)

bcs_condition = ((tableP['TCV']>= 25000) & (tableP['AS Offer Type']=='BCS'))
htec_condition = ((tableP['TCV']>= 20000) & (tableP['AS Offer Type']=='HTEC'))
sspt_condition = ((tableP['TCV']>= 20000) & (tableP['AS Offer Type']=='Solution Support'))
st_condition = ((tableP['TCV']>= 5000) & (tableP['AS Offer Type']=='SuccessTracks'))
sntc_condition = ((tableP['TCV']>= 15000) & (tableP['AS Offer Type']=='SNTC'))

In [144]:
tableP = tableP.loc[bcs_condition | htec_condition | sspt_condition | st_condition | sntc_condition]

In [145]:
table = pd.concat([tableN,tableP])

In [146]:
# table = table.loc[~(table['Deal ID']==-9999)]

In [147]:
table.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Deal ID',
       'FINBI Partner', 'AS Offer Type', 'CX Product Category',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Service Source',
       'Squeezed', 'Prepend', 'FakeID', 'Prehash_Entity', 'ACV', 'TCV', 'Ind'],
      dtype='object')

In [148]:
table.sort_values(by=['CAV Name', 'GUID', 'GUID Name','FINBI Deal ID', 'Fiscal Week ID', 'AS Offer Type', 'Ind'], inplace=True)

In [149]:
table.loc[table['CX Product Category'].str.lower()=='successtracks']

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Fiscal Week ID,ReportWeek,Service Source,Squeezed,Prepend,FakeID,Prehash_Entity,ACV,TCV,Ind
77465,Americas,US COMMERCIAL,COMMERCIAL WEST AREA,PACIFIC WEST COMMERCIAL OPERATION,533239,ALLEGIANT TRAVEL COMPANY,41323642,ALLEGIANT TRAVEL COMPANY,43648027,UNIVERSAL E-BUSINESS SOLUTIONS DBA UNIVERSAL C...,...,2022091,9,NEW,ALLEGIANTTRAVELCOMPANY,AL,41323642,AL41323642,"5,125.98","14,119.74",P
77466,Americas,US COMMERCIAL,COMMERCIAL WEST AREA,PACIFIC WEST COMMERCIAL OPERATION,533239,ALLEGIANT TRAVEL COMPANY,41323642,ALLEGIANT TRAVEL COMPANY,43648027,UNIVERSAL E-BUSINESS SOLUTIONS DBA UNIVERSAL C...,...,2022091,9,RENEW,ALLEGIANTTRAVELCOMPANY,AL,41323642,AL41323642,"22,665.55","62,228.05",P
60805,Americas,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,221089,ALLETE INC,15244,ALLETE INC,43528128,CDW,...,2022091,9,RENEW,ALLETEINC,AL,15244,AL15244,"43,897.64","185,038.12",P
26775,Americas,GLOBAL ENTERPRISE SEGMENT,GES PREMIER,GLL_P_A_AMEX,100688,AMERICAN EXPRESS COMPANY,15497,AMERICAN EXPRESS COMPANY,55899463,UNKNOWN,...,2022091,9,NEW,AMERICANEXPRESSCOMPANY,AM,15497,AM15497,"49,613.71","144,947.68",P
26776,Americas,GLOBAL ENTERPRISE SEGMENT,GES PREMIER,GLL_P_A_AMEX,100688,AMERICAN EXPRESS COMPANY,15497,AMERICAN EXPRESS COMPANY,55899463,UNKNOWN,...,2022091,9,RENEW,AMERICANEXPRESSCOMPANY,AM,15497,AM15497,"1,766,897.35","5,094,762.48",P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19848,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,GREAT_NORTH_OPERATION,103201,US BANCORP,13959,US BANCORP,44197795,UNKNOWN,...,2022091,9,UNKNOWN,USBANCORP,US,13959,US13959,"2,242,199.93","6,700,100.62",P
108137,Americas,US PS Market Segment,SLED-East Area,SLED-Centraleast Operation,471525,VIRGINIA COMMONWEALTH UNIVERSITY,11769,COMMONWEALTH OF VIRGINIA,54186355,"SYCOM TECHNOLOGIES, L.L.C. (VIRGINIA)",...,2022091,9,NEW,VIRGINIACOMMONWEALTHUNIVERSITY,VI,11769,VI11769,"3,523.43","17,617.16",P
119777,Americas,US PS Market Segment,SLED-West Area,SLED-West By Midwest Operation,675828,WISCONSIN TECHNICAL COLLEGE SYSTEM BOARD,3957523,WISCONSIN TECHNICAL COLLEGE SYSTEM,55734127,HEARTLAND BUSINESS SYSTEMS,...,2022092,10,NEW,WISCONSINTECHNICALCOLLEGESYSTEMBOARD,WI,3957523,WI3957523,"67,447.46","337,237.29",P
119780,Americas,US PS Market Segment,SLED-West Area,SLED-West By Midwest Operation,675828,WISCONSIN TECHNICAL COLLEGE SYSTEM BOARD,3957523,WISCONSIN TECHNICAL COLLEGE SYSTEM,55797945,HEARTLAND BUSINESS SYSTEMS,...,2022092,10,NEW,WISCONSINTECHNICALCOLLEGESYSTEMBOARD,WI,3957523,WI3957523,"14,978.25","74,891.25",P


In [150]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2288 entries, 18715 to 74063
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Sales Level 1        2288 non-null   object 
 1   Sales Level 2        2288 non-null   object 
 2   Sales Level 3        2288 non-null   object 
 3   Sales Level 4        2288 non-null   object 
 4   CAV ID               2288 non-null   int64  
 5   CAV Name             2288 non-null   object 
 6   GUID                 2288 non-null   int64  
 7   GUID Name            2288 non-null   object 
 8   FINBI Deal ID        2288 non-null   int64  
 9   FINBI Partner        2288 non-null   object 
 10  AS Offer Type        2288 non-null   object 
 11  CX Product Category  2288 non-null   object 
 12  Fiscal Quarter ID    2288 non-null   object 
 13  Fiscal Week ID       2288 non-null   int64  
 14  ReportWeek           2288 non-null   int64  
 15  Service Source       2288 non-nul

In [151]:
# ro.rename(columns = {'Customer Name':'Compass GUID Name', 'Deal ID':'Compass Deal ID'}, inplace=True)
ro

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
1,118239,11438,AMERICAS,US PS MARKET SEGMENT,SLED-WEST AREA,SLED-CENTRAL OPERATION,TEXAS BUILDING AND PROCUREMENT COMMISSION,44358639,REACTIVE,Mark Vodka,TE11438,2022-04-08,NaN,NaN,FIELD,NaN,NaT
2,117567,444444,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
3,117069,16288,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
4,116418,4471862,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,HENDRICKS,55099391,REACTIVE,Term,HE4471862,2022-04-08,Emailed Recommendations,"BCS,BRW,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
5,116075,6832603,APJC,ASEAN_AREA,ASEAN_SMALL,ASEAN_SMALL_SGP,DBS CHINA COMPANY LTD,<NA>,REACTIVE,Houman Asefi,DB6832603,2022-04-07,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",FIELD,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13684,107566,10368781,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
13685,107567,320903967,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
13686,107568,3773236,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
13687,107569,2134315,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [152]:
ro = ro.loc[(ro['Readout_Date'].notnull()) & (ro['Sales Level 1']=='AMERICAS') & ~(ro['Readout Delivery Status'].isin(['Declined','Postponed']))]

In [153]:
ro.loc[ro['CONCIERGE_INTERNAL_READOUT_DT'].notnull()]

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
3,117069,16288,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
627,115703,224733949,AMERICAS,US COMMERCIAL,COMMERCIAL EAST AREA,MID-ATLANTIC COMMERCIAL OPERATION,RAPPAHANNCOCK ELECTRICAL,<NA>,REACTIVE,Adrian Machado,RA224733949,2022-03-18,Complete,"BCS,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SSPT,ST",FIELD,admachad@cisco.com,2022-03-18
815,115940,45930,AMERICAS,US COMMERCIAL,COMMERCIAL WEST AREA,PACIFIC WEST COMMERCIAL OPERATION,PHOENIX CHILDRENS HOSPITAL,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Juan Prieto,PH45930,2022-04-08,NaN,NaN,COVERAGE AT RENEWAL,glcanale@cisco.com,2022-03-30
842,116034,4456229,AMERICAS,US COMMERCIAL,COMMERCIAL EAST AREA,SOUTH EAST COMMERCIAL OPERATION,INVESCO,<NA>,REACTIVE,Adrian Machado,IN4456229,2022-03-28,Complete,"BCS,DNA,EC-AM,IRR,PR,SNTC,SSPT,ST",FIELD,admachad@cisco.com,2022-03-28
843,116035,50403,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,HEARTLAND_OPERATION,SANFORD HEALTH,44323285,REACTIVE,Peter Wittenstrom,SA50403,2022-04-13,NaN,NaN,FIELD,pwittens@cisco.com,2022-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11689,104859,164032320,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO ENTERPRISE,GRUPO COPPEL SA DE CV,43485790,REACTIVE,Adrian Machado,GR164032320,2021-11-16,Complete,"BCS,DNA,EC-HTOM,EC-AM,IRR,SNTC,ST,SSPT",FIELD,admachad@cisco.com,2021-11-16
11690,104871,2169777,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO PUBLIC SECTOR,CONSEJO DE LA JUDICATURA FEDERAL,43513084,REACTIVE,Adrian Machado,CO2169777,2021-11-18,Complete,"BCS,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,ST,SSPT",FIELD,admachad@cisco.com,2021-11-18
11695,106628,37106,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,HEARTLAND_OPERATION,MERCY HOSPITAL,<NA>,REACTIVE,Mark Vodka,ME37106,2021-12-02,Complete,"BCS,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,ST",FIELD,mvodka@cisco.com,2021-12-02
12233,105772,2169777,AMERICAS,LATIN AMERICA,MEXICO-,MEXICO PUBLIC SECTOR,BENEMERITA UNIVERSIDAD AUTONOMA DE PUEBLA,55456804,REACTIVE,Miguel Mora,BE2169777,2022-02-16,Complete,"SNTC,SSPT,ST,EC-KT",FIELD,admachad@cisco.com,2022-03-29


In [154]:
today = dt.datetime.now()

In [155]:
today.date()

datetime.date(2022, 4, 11)

In [156]:
cal

,date,fiscalyear,daynumber,fiscalquarter,days_in_quarter,quarterdays_end,quarterdays_begin,month_of_quarter,month_of_fiscalyear,week_of_fiscalyear,week_of_fiscalquarter,week_of_fiscalmonth,finbi_fiscal_quarter_id,finbi_fiscal_period_id,finbi_fiscal_week_id,report_week_of_quarter
0,1999-08-01,2000,1,1,91,91,0.00,1,1,1,1,1,2000Q1,200001,2000011,1
4,1999-08-02,2000,2,1,91,91,0.00,1,1,1,1,1,2000Q1,200001,2000011,1
8,1999-08-03,2000,3,1,91,91,0.00,1,1,1,1,1,2000Q1,200001,2000011,1
12,1999-08-04,2000,4,1,91,91,0.00,1,1,1,1,1,2000Q1,200001,2000011,1
16,1999-08-05,2000,5,1,91,91,0.00,1,1,1,1,1,2000Q1,200001,2000011,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40891,2027-07-27,2027,367,4,91,371,280.00,3,12,53,13,5,2027Q4,202712,2027125,13
40895,2027-07-28,2027,368,4,91,371,280.00,3,12,53,13,5,2027Q4,202712,2027125,13
40899,2027-07-29,2027,369,4,91,371,280.00,3,12,53,13,5,2027Q4,202712,2027125,13
40903,2027-07-30,2027,370,4,91,371,280.00,3,12,53,13,5,2027Q4,202712,2027125,13


In [157]:
def get_minimum_date_from_weekid(weekid):
    return cal.loc[cal['finbi_fiscal_week_id']==weekid, 'date'].max()

In [158]:
fiscalweeks = list(table['Fiscal Week ID'].unique())

In [159]:
fiscalweeks.sort()

In [160]:
cal = cal.loc[cal['finbi_fiscal_week_id'].isin(fiscalweeks)]

In [161]:
### Add a time delta function here to get back the 30 days PRIOR to this min date for each fiscal week
min_dates_dict = {}
for x in fiscalweeks:
    min_dates_dict[x] = get_minimum_date_from_weekid(x)

min_dates_dict

{2022091: '2022-04-02', 2022092: '2022-04-09'}

In [162]:
must_be_prior_to_dict = {}
for k, v in min_dates_dict.items():
    must_be_prior_to_dict[k] = (pd.to_datetime(v) + dt.timedelta(days = -30))

In [163]:
must_be_prior_to_dict

{2022091: Timestamp('2022-03-03 00:00:00'),
 2022092: Timestamp('2022-03-10 00:00:00')}

In [164]:
ro

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
1,118239,11438,AMERICAS,US PS MARKET SEGMENT,SLED-WEST AREA,SLED-CENTRAL OPERATION,TEXAS BUILDING AND PROCUREMENT COMMISSION,44358639,REACTIVE,Mark Vodka,TE11438,2022-04-08,NaN,NaN,FIELD,NaN,NaT
2,117567,444444,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
3,117069,16288,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
4,116418,4471862,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,HENDRICKS,55099391,REACTIVE,Term,HE4471862,2022-04-08,Emailed Recommendations,"BCS,BRW,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
31,115028,3255593,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-CENTRALEAST OPERATION,VA-COUNTY OF PRINCE WILLIAM SCHOOL DISTRICT,<NA>,COVERAGE PROGRAM - TELEMETRY,Emma Baird,VA3255593,2022-03-28,Complete,"SNTC,SSPT,BCS,DNA,ST,PR,EC-AM,EC-HTOM,EC-KT,IRR",COVERAGE PROGRAM,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13684,107566,10368781,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
13685,107567,320903967,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
13686,107568,3773236,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
13687,107569,2134315,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [165]:
ro['Readout_Date'] = pd.to_datetime(ro['Readout_Date'])
ro['CONCIERGE_INTERNAL_READOUT_DT'] = pd.to_datetime(ro['CONCIERGE_INTERNAL_READOUT_DT'])

<ipython-input-165-be40e7ad5c40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro['Readout_Date'] = pd.to_datetime(ro['Readout_Date'])
<ipython-input-165-be40e7ad5c40>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro['CONCIERGE_INTERNAL_READOUT_DT'] = pd.to_datetime(ro['CONCIERGE_INTERNAL_READOUT_DT'])


In [166]:
ro.loc[ro['Readout_Date'].isnull()][['Request ID','Readout_Date']]

,Request ID,Readout_Date


In [167]:
ro['Readout_Date'] = ro['Readout_Date'].astype(str).replace('-','')

<ipython-input-167-b82cb0de2393>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ro['Readout_Date'] = ro['Readout_Date'].astype(str).replace('-','')


In [168]:
ro.loc[ro['Readout_Date'].notnull()]

,Request ID,GUID,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Compass GUID Name,Compass Deal ID,Campaign,Compass Advisor,Prehash_Entity,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT
1,118239,11438,AMERICAS,US PS MARKET SEGMENT,SLED-WEST AREA,SLED-CENTRAL OPERATION,TEXAS BUILDING AND PROCUREMENT COMMISSION,44358639,REACTIVE,Mark Vodka,TE11438,2022-04-08,NaN,NaN,FIELD,NaN,NaT
2,117567,444444,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET NAMED,BLUE RIDGE COMMUNICATIONS INC,<NA>,COVER THE UNCOVERED - COVERAGE AT RENEWAL,Term,BL444444,2022-04-12,NaN,NaN,COVERAGE AT RENEWAL,NaN,NaT
3,117069,16288,AMERICAS,US PS MARKET SEGMENT,PS FEDERAL AREA,FED-NATIONAL SECURITY OPERATION,MITRE,<NA>,REACTIVE,Emma Baird,MI16288,2022-04-19,NaN,NaN,FIELD,ebaird@cisco.com,2022-04-18
4,116418,4471862,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,HENDRICKS,55099391,REACTIVE,Term,HE4471862,2022-04-08,Emailed Recommendations,"BCS,BRW,EC-KT,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
31,115028,3255593,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-CENTRALEAST OPERATION,VA-COUNTY OF PRINCE WILLIAM SCHOOL DISTRICT,<NA>,COVERAGE PROGRAM - TELEMETRY,Emma Baird,VA3255593,2022-03-28,Complete,"SNTC,SSPT,BCS,DNA,ST,PR,EC-AM,EC-HTOM,EC-KT,IRR",COVERAGE PROGRAM,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13684,107566,10368781,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SYNCRO CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,SY10368781,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,NaN,NaT
13685,107567,320903967,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,ENCOMPASS HEALTH CORPORATION,<NA>,REACTIVE,Sangeetha Sampath,EN320903967,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,NaN,NaT
13686,107568,3773236,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,SLIDELL MEMORIAL HOSPITAL,<NA>,REACTIVE,Sangeetha Sampath,SL3773236,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT
13687,107569,2134315,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,MILLIPORE SIGMA,<NA>,REACTIVE,Sangeetha Sampath,MI2134315,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,NaN,NaT


In [169]:
# ro['TimeDelta'] = ro['Date Readout Done'].apply(lambda x: today.date() - x)

In [170]:
MaxList = pd.DataFrame(ro.groupby('Compass GUID Name')['Readout_Date'].max())

In [171]:
MaxList.reset_index(inplace=True)

In [172]:
MaxList

,Compass GUID Name,Readout_Date
0,3M COMPANY,2022-04-07
1,3M US,2022-04-06
2,66 FEDERAL CREDIT UNION,2022-03-16
3,A123 SYSTEMS INC,2021-12-14
4,AAA COOPER TRANSPORTATION,2021-12-08
...,...,...
3116,ZIONS BANCORP,2021-12-02
3117,ZOETIS INC,2022-01-19
3118,ZOTEC PARTNERS,2022-02-28
3119,ZS ASSOCIATES INC,2022-04-04


In [173]:
output = pd.merge(ro,MaxList, how='inner', left_on = 'Compass GUID Name', right_on='Compass GUID Name').drop(columns = ['Readout_Date_y'])#.rename(columns={'Date Readout Done_x':'Date Readout Done'}, inplace=True)

In [174]:
output.rename(columns={'Readout_Date_x':'Readout_Date'}, inplace=True)

In [175]:
output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5203 entries, 0 to 5202
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Request ID                     5203 non-null   object        
 1   GUID                           5203 non-null   Int64         
 2   Sales Level 1                  5203 non-null   object        
 3   Sales Level 2                  5203 non-null   object        
 4   Sales Level 3                  5203 non-null   object        
 5   Sales Level 4                  5203 non-null   object        
 6   Compass GUID Name              5203 non-null   object        
 7   Compass Deal ID                1667 non-null   object        
 8   Campaign                       5203 non-null   object        
 9   Compass Advisor                5202 non-null   object        
 10  Prehash_Entity                 5203 non-null   object        
 11  Readout_Date     

In [176]:
def set_threshold_date(finbi_week_id):
    return must_be_prior_to_dict.get(finbi_week_id)

In [177]:
table['Threshold Date'] = table.apply(lambda x: set_threshold_date(x['Fiscal Week ID']), axis=1)

In [178]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2288 entries, 18715 to 74063
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Sales Level 1        2288 non-null   object        
 1   Sales Level 2        2288 non-null   object        
 2   Sales Level 3        2288 non-null   object        
 3   Sales Level 4        2288 non-null   object        
 4   CAV ID               2288 non-null   int64         
 5   CAV Name             2288 non-null   object        
 6   GUID                 2288 non-null   int64         
 7   GUID Name            2288 non-null   object        
 8   FINBI Deal ID        2288 non-null   int64         
 9   FINBI Partner        2288 non-null   object        
 10  AS Offer Type        2288 non-null   object        
 11  CX Product Category  2288 non-null   object        
 12  Fiscal Quarter ID    2288 non-null   object        
 13  Fiscal Week ID       2288 no

In [179]:
compass_exists_table = pd.merge(table,output, how='left', left_on=['Prehash_Entity'], right_on = ['Prehash_Entity'])

In [180]:
compass_exists_table['Compass Exists?'] = ''
compass_exists_table['Compass Claim?'] = ''
compass_exists_table['Notes'] = ''

In [181]:
compass_exists_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3166 entries, 0 to 3165
Data columns (total 43 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Sales Level 1_x                3166 non-null   object        
 1   Sales Level 2_x                3166 non-null   object        
 2   Sales Level 3_x                3166 non-null   object        
 3   Sales Level 4_x                3166 non-null   object        
 4   CAV ID                         3166 non-null   int64         
 5   CAV Name                       3166 non-null   object        
 6   GUID_x                         3166 non-null   int64         
 7   GUID Name                      3166 non-null   object        
 8   FINBI Deal ID                  3166 non-null   int64         
 9   FINBI Partner                  3166 non-null   object        
 10  AS Offer Type                  3166 non-null   object        
 11  CX Product Catego

### Create the joined Compass Exists table here

In [182]:
# compass_exists_table = pd.merge(compass_exists_table, services_source, how='left', left_on = 'FINBI Deal ID', right_on = 'Deal ID')

In [183]:
compass_exists_table['Readout_Date'] = pd.to_datetime(compass_exists_table['Readout_Date'])

In [184]:
compass_exists_table['Threshold Date'] = pd.to_datetime(compass_exists_table['Threshold Date'])

In [185]:
condition1 = ((compass_exists_table['CONCIERGE_INTERNAL_READOUT_DT'] <= compass_exists_table['Threshold Date']) | (compass_exists_table['Readout_Date'] <= compass_exists_table['Threshold Date']))  #
condition2 = ((compass_exists_table['Readout_Date'].isnull()) & (compass_exists_table['FINBI Deal ID']==-9999))

In [186]:
compass_exists_table = compass_exists_table.loc[condition1 & ~condition2] #[['FINBI Deal ID','Threshold Date','Date Readout Done', 'Compass Deal ID', 'Deal ID']]

In [187]:
# compass_exists_table = compass_exists_table.loc[condition1 & ~condition2] #[['FINBI Deal ID','Threshold Date','Date Readout Done', 'Compass Deal ID', 'Deal ID']]

In [188]:
compass_exists_table.rename(columns={'Sales Level 1_x':'Sales Level 1',
                                     'Sales Level 2_x':'Sales Level 2',
                                     'Sales Level 3_x':'Sales Level 3',
                                     'Sales Level 4_x':'Sales Level 4',
                                     'GUID_x':'GUID',
                                     'GUID_y':'Compass GUID'
                                     }, inplace=True)

In [189]:
compass_exists_table.drop(columns=(['Squeezed', 'Prepend', 'FakeID', 'Prehash_Entity','Sales Level 1_y',]), inplace=True)

In [190]:
# compass_exists_table.drop(columns=(['CONCIERGE_REQUESTED_FLAG','CONCIERGE_INTERNAL_RESULT']), inplace=True)

In [191]:
compass_exists_table

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Deal ID,FINBI Partner,...,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT,Compass Exists?,Compass Claim?,Notes
13,Americas,US COMMERCIAL,COMMERCIAL EAST AREA,SOUTH EAST COMMERCIAL OPERATION,479713,ACI WORLDWIDE INC,187793,ACI WORLDWIDE INC,54351297,THE CUMBERLAND GROUP,...,Juan Cabrera,2021-12-16,Complete,SNTC,FIELD,NaN,NaT,,,
15,Americas,US COMMERCIAL,COMMERCIAL EAST AREA,SOUTH EAST COMMERCIAL OPERATION,479713,ACI WORLDWIDE INC,187793,ACI WORLDWIDE INC,54351297,THE CUMBERLAND GROUP,...,Barbara Brown,2021-06-14,Complete,"BCS, BRW, EC-AM, EC-HTOM, IRR, PR, SNTC, SSPT",CXIB,NaN,NaT,,,
21,Americas,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,SOUTHEAST OP,103157,ADVENTHEALTH,34694,ADVENTIST HEALTH SYSTEM SUNBELT INC,20188711,CDW,...,Alicia Biersteker,2021-11-18,Complete,"BCS,BRW,ST,IRR,PR,SNTC,SSPT,EC-AM,EC-HTOM,EC-KT",TARGETING,NaN,NaT,,,
22,Americas,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,SOUTHEAST OP,103157,ADVENTHEALTH,34694,ADVENTIST HEALTH SYSTEM SUNBELT INC,20188711,CDW,...,Alicia Biersteker,2021-11-18,Complete,"BCS,BRW,ST,IRR,PR,SNTC,SSPT,EC-AM,EC-HTOM,EC-KT",TARGETING,NaN,NaT,,,
23,Americas,US COMMERCIAL,COMMERCIAL WEST AREA,GOLDEN WEST COMMERCIAL OPERATION,103034,ADVENTIST HEALTH WEST,36063,ADVENTIST HEALTH SYSTEMWEST,54118622,CDW,...,Mark Vodka,2021-06-29,Complete,"BCS, BRW, EC-AM, EC-HTOM, EC-KT, IRR, PR, SNTC...",CXIB,NaN,NaT,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,Americas,GLOBAL ENTERPRISE SEGMENT,GES EAST,ATLANTIC ENTERPRISE OPERATION,108051,WORLD BANK GROUP,29886,INTERNATIONAL BANK FOR RECONSTRUCTION & DEVELO...,54331436,PRESIDIO NETWORKED SOLUTIONS,...,Adrian Machado,2022-01-07,Complete,"BCS,DNA,EC-HTOM,EC-KT,IRR,SNTC,SSPT,ST",TARGETING,NaN,NaT,,,
3154,Americas,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,SOUTHEAST OP,111944,WYNDHAM WORLDWIDE CORPORATION,41324904,WYNDHAM HOTELS & RESORTS INC,55537260,"Converge Acquisition, LLC",...,Sara Villamarin,2021-06-15,Complete,"EC-AM, EC-HTOM, EC-KT, SNTC, SSPT, ST, SW-E/P",FIELD,NaN,NaT,,,
3155,Americas,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,GREAT_NORTH_OPERATION,107224,XCEL ENERGY INC,4743798,XCEL ENERGY INC,53917184,"AHEAD, Inc.",...,Peter Wittenstrom,2022-01-12,Complete,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,PR,SNTC,SSPT,ST",FIELD,NaN,NaT,,,
3156,Americas,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,166763,"XENOTECH, L.L.C.",35192135,"XENOTECH, L.L.C.",-9999,CDW,...,Sangeetha Sampath,2021-12-21,Complete,"SSPT,SNTC",FIELD,NaN,NaT,,,


In [192]:
compass_exists_table.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Deal ID',
       'FINBI Partner', 'AS Offer Type', 'CX Product Category',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Service Source',
       'ACV', 'TCV', 'Ind', 'Threshold Date', 'Request ID', 'Compass GUID',
       'Sales Level 2_y', 'Sales Level 3_y', 'Sales Level 4_y',
       'Compass GUID Name', 'Compass Deal ID', 'Campaign', 'Compass Advisor',
       'Readout_Date', 'Readout Delivery Status', 'Items Recommended',
       'Campaign Type', 'CONCIERGE_ADVISOR_ASSIGNED',
       'CONCIERGE_INTERNAL_READOUT_DT', 'Compass Exists?', 'Compass Claim?',
       'Notes'],
      dtype='object')

In [193]:
# compass_exists_table = compass_exists_table[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Deal ID', 
#                                              'FINBI Partner', 'AS Offer Type', 'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID','ReportWeek', 'ACV', 'TCV', 'Ind', 
#                                              'Request ID', 'Compass GUID Name', 'Compass Deal ID', 'Compass Campaign Name', 'Threshold Date', 'Date Readout Done','Compass Advisor',
#                                              'Readout Delivery Status', 'Items Recommended', 'Campaign Type', 'IndexID', 'Compass Exists?', 'Compass Claim?', 'Notes', 'Service Source'
#                                             ]]

In [194]:
product_types = {'BCS 2.x':'BCS 2',
                 'BCS 2.0':'BCS 2',
                 'BCS 3.x':'BCS 3',
                 'BCS 3.0':'BCS 3',
                 'HTEC':'HTEC',
                 'Expert Care':'HTEC',
                 'Level 1':'ST',
                 'Level 2':'ST',
                 'Solution Support':'SSPT',
                 'SuccessTracks':'ST',
                 'Campus Network':'ST',
                 'Unknown':'UNK',
                 'SNTC':'SNTC'}

In [195]:
compass_exists_table.rename(columns={'AS Offer Type':'Product Type'}, inplace=True)

In [196]:
compass_exists_table['Compass Deal ID'] = compass_exists_table['Compass Deal ID'].apply(convert_nan_to_int)

In [197]:
compass_exists_table['Product Type'] = compass_exists_table['CX Product Category'].apply(lambda x: product_types.get(x))

In [198]:
compass_exists_table.sort_values(by=['GUID Name', 'CAV Name', 'Fiscal Quarter ID', 'ReportWeek', 'Sales Level 1','Sales Level 2','Sales Level 3','Product Type','FINBI Deal ID', 'TCV'], inplace=True)

In [199]:
def apply_targeting_campaign_flag(compass_campaign_name):
    if pd.notnull(compass_campaign_name) and ('target' in compass_campaign_name.lower() or compass_campaign_name.upper()=='SUCCESS TRACKS FOR CAMPUS'):
        return True
    else:
        return False

In [200]:
compass_exists_table['Target Campaign Flag'] = compass_exists_table.apply(lambda x: apply_targeting_campaign_flag(x['Campaign']), axis=1)

In [201]:
compass_exists_table['DupeCheck'] =  compass_exists_table.duplicated(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name','GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID','CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID','ReportWeek', 'Product Type', 'TCV'], keep=False)

In [202]:
# compass_exists_table.rename(columns={'Services Source':'Service Source'}, inplace=True)

In [203]:
compass_exists_table = compass_exists_table[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Partner', 'FINBI Deal ID',
                                             'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID', 'ReportWeek', 'Product Type', 'TCV', 'ACV', 'Ind', 'DupeCheck', 
                                             'Request ID', 'Compass GUID','Compass GUID Name', 'Compass Deal ID', 'Campaign', 'Threshold Date', 'Readout_Date',
                                             'Compass Advisor', 'Readout Delivery Status', 'Items Recommended', 'Campaign Type', 'Service Source', 'CONCIERGE_ADVISOR_ASSIGNED', 'CONCIERGE_INTERNAL_READOUT_DT','Target Campaign Flag', 'Compass Exists?', 
                                             'Compass Claim?', 'Notes', 
                                            ]]

In [204]:
compass_exists_table = compass_exists_table.loc[(compass_exists_table['Readout Delivery Status']!= 'Declined')] #(compass_exists_table['Service Source'].notnull()) | 

In [205]:
compass_exists_table.shape

(1284, 37)

In [206]:
table.rename(columns={'AS Offer Type':'Product Type'}, inplace=True)

In [207]:
table['Product Type'] = table['CX Product Category'].apply(lambda x: product_types.get(x))

In [208]:
table['Product Type'].unique()

array(['SNTC', 'SSPT', 'ST', 'BCS 2', 'HTEC', 'BCS 3'], dtype=object)

In [209]:
# success.loc[(success['Fiscal Quarter ID']=='2021Q2') & (success['Fiscal Week ID'] >= 2021053)]

In [210]:
# compass_exists_table.loc[compass_exists_table['Service Source'].isnull()]['FINBI Deal ID'].nunique()/

In [211]:
with pd.ExcelWriter(fin_path + "\\" + endfilename) as writer:  #endfilename
    df.to_excel(writer, sheet_name = 'RawData', index=False)
    table.to_excel(writer, sheet_name = 'Pivot', index = False)
    compass_exists_table.to_excel(writer, sheet_name = "Compass Exists", index=False)